In [1]:
#the imports
import cv2
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
import util
import time
#import model
#import that model:
import import_ipynb
import model

import os
#from argparse import ArgumentParser

2023-04-14 01:18:39.014457: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


importing Jupyter notebook from model.ipynb


In [2]:
#The parameters
imbatch_read     = util.imbatch_read
img_write        = util.img_write
pickup_list      = util.pickup_list
ls_files_to_json = util.ls_files_to_json

build_generator     = model.build_generator

# args = build_parser().parse_args()

MODEL_SAVE_PATH = 'available_models/Jason_Test_Model/'

BATCH_SIZE  = 1
FEED_SIZE   = 256

IMGSRC_PATH = 'test_images/'
NOISE_RATE  = 0

#use_cpu
USE_CPU = False
#out_path
OUT_PATH = 'rendered_images/'

if (not os.path.isdir(OUT_PATH)):
	os.makedirs(OUT_PATH)

In [4]:
DEVICE = ''
if USE_CPU =='TRUE':
	DEVICE = '/cpu:0'

input_ls = ls_files_to_json(IMGSRC_PATH, ext=['png', 'bmp', 'jpg', 'jpeg'])
CNT = len(input_ls)

gpu_options = tf.GPUOptions(allow_growth=True)

with tf.device(DEVICE), tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
	input_r = tf.placeholder(tf.float32, shape=[BATCH_SIZE, FEED_SIZE, FEED_SIZE, 3], name='inpr')
	g_state = build_generator(input_r, name='generator')
	g_var_ls = tf.trainable_variables(scope='generator')
	sess.run(tf.global_variables_initializer())
	saver = tf.train.Saver(g_var_ls)
	chkpt_fname = tf.train.latest_checkpoint(MODEL_SAVE_PATH)
	saver.restore(sess, chkpt_fname)
	# Warm up network and test...
	noise = np.random.normal(0, 1, size=(BATCH_SIZE, FEED_SIZE, FEED_SIZE, 3)).astype(np.float32)
	sess.run(g_state, feed_dict={input_r: noise})
	# Begin...
	total = int(CNT / BATCH_SIZE) + (1 if (CNT % BATCH_SIZE) != 0 else 0)
	for offset in range(total):
		sub_ls  = pickup_list(input_ls, BATCH_SIZE, offset * BATCH_SIZE)
		sub_img, sls = imbatch_read(IMGSRC_PATH, sub_ls, (FEED_SIZE, FEED_SIZE))
		if NOISE_RATE != 0:
			noise = np.random.normal(0, 1,
				size=(BATCH_SIZE, FEED_SIZE, FEED_SIZE, 3)).astype(np.float32)
			sub_img = sub_img + noise * NOISE_RATE 
		time_start = time.time()
		render_batch = sess.run(g_state, feed_dict={input_r: sub_img})
		print('Processing: %d/%d, network dataflow time: %f'%(offset + 1, total, time.time()-time_start))
		for i in range(len(sls)):
			img_write(render_batch[i],
					OUT_PATH+'/%d_%d.jpg'%(offset, i),
				sls[i])# (FEED_SIZE, FEED_SIZE)) #

2023-04-14 01:19:42.374018: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-14 01:19:42.374822: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-04-14 01:19:42.582675: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Restoring parameters from available_models/Jason_Test_Model/model-2


2023-04-14 01:19:42.999446: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-14 01:19:43.055917: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Processing: 1/4, network dataflow time: 0.030296
Processing: 2/4, network dataflow time: 0.029673
Processing: 3/4, network dataflow time: 0.027872
Processing: 4/4, network dataflow time: 0.027205
